In [1]:
! npm install localtunnel


added 22 packages in 3s

3 packages are looking for funding
  run `npm fund` for details


npm notice
npm notice New major version of npm available! 10.9.2 -> 11.3.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.3.0
npm notice To update run: npm install -g npm@11.3.0
npm notice


In [10]:
%%writefile app_Valencia.py 
#####################################################
#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import plotly.figure_factory as ff

######################################################
#Definimos la instancia
@st.cache_resource
######################################################
#Creamos la función de carga de datos
def load_data():
    df = pd.read_csv("Valencia.csv")

    # Filtrar precios razonables (por debajo de 500)
    df = df[df['price'] <= 700]

    # Selecciono columnas relevantes, ya SIN id, name, neighbourhood_group
    selected_cols = [
        'host_id', 'host_name', 'neighbourhood', 'host_is_superhost',
        'room_type', 'accommodates', 'amenities', 'price', 'minimum_nights',
        'number_of_reviews', 'reviews_per_month', 'host_identity_verified',
        'availability_365', 'host_response_time', 'bathrooms_text', 'bathrooms',
        'bedrooms', 'beds', 'has_availability', 'instant_bookable'
    ]
    df = df[selected_cols]

    # Columnas numéricas
    numeric_df = df.select_dtypes(include=['float', 'int'])
    numeric_cols = numeric_df.columns

    # Columnas de texto/categóricas
    text_df = df.select_dtypes(include=['object'])
    text_cols = text_df.columns

    # Categorías únicas de 'room_type'
    unique_categories_room = df['room_type'].unique()

    return df, numeric_cols, text_cols, unique_categories_room, numeric_df

###############################################################################
#Cargo los datos obtenidos de la función "load_data"
df, numeric_cols, text_cols, unique_categories_room, numeric_df = load_data()
###############################################################################
# ESTILOS PERSONALIZADOS
st.markdown("""
    <style>
        .main {
            background-color: #FDF6E3;
            font-family: 'Segoe UI', sans-serif;
        }

        .stApp {
            background-image: linear-gradient(to bottom right, #FDF6E3, #FFFFFF);
        }

        .stSidebar {
            background-color: #F7941D;
            color: white;
        }

        .stSidebar h1, .stSidebar h2, .stSidebar h3 {
            color: white;
        }

        .stButton>button {
            border-radius: 12px;
            font-weight: bold;
        }

        .stSidebar button {
            background-color: #ffb300;
            color: white;
            border: none;
            border-radius: 12px;
            padding: 0.6rem 1.2rem;
            margin-top: 0.5rem;
            font-size: 1rem;
            font-weight: bold;
            transition: background-color 0.3s ease, transform 0.2s ease;
        }

        .stSidebar button:hover {
            background-color: #E63946;
            transform: scale(1.05);
        }

        h1, h2, h3 {
            color: #E63946;
            font-weight: bold;
            text-shadow: 1px 1px 2px rgba(0, 0, 0, 0.1);
            padding-bottom: 0.5rem;
            border-bottom: 3px solid #F7941D;
            margin-bottom: 1rem;
            text-align: center;
        }

        h1 {
            font-size: 2.5rem;
        }

        h2 {
            font-size: 2rem;
        }

        h3 {
            font-size: 1.6rem;
        }

        .block-container {
            padding-top: 3rem;
        }
    </style>
""", unsafe_allow_html=True)




# LOGOS EN TODAS LAS PÁGINAS
with st.sidebar:
    st.title("VALENCIA, ESPAÑA")

    st.markdown("""
        <div style="display: flex; justify-content: center;">
            <img src="https://upload.wikimedia.org/wikipedia/commons/6/69/Airbnb_Logo_B%C3%A9lo.svg" width="160">
        </div>
    """, unsafe_allow_html=True)

# MENÚ PRINCIPAL
View = st.sidebar.selectbox(
    label="Selecciona una sección:", 
    options=["Inicio", "Análisis Categórico y Numérico", "Análisis de Correlaciones", "Regresión Logística"]
)

# ----------------------------------------------------------------------------------------------------
# VISTA INICIO 
if View == "Inicio":
    st.title("Valencia y Airbnb: ¡Una combinación perfecta!")
    st.divider()

    col1, col2 = st.columns([2, 2])

    with col1:
        st.subheader("Explora la magia de Valencia a través de los datos.")
        st.markdown("""
        Valencia, la joya del Mediterráneo, fusiona historia, innovación y cultura vibrante.  

        A través de este panel interactivo podrás:
        - Analizar tipos de alojamiento más populares
        - Descubrir tendencias de precios
        - Explorar disponibilidad anual
        - Investigar variables que afectan el éxito de los anuncios

        **¡Descubre Valencia desde una nueva perspectiva basada en datos!**  
        """)

        with col2:
            st.markdown(
                """
                <div style="text-align: center;">
                    <img src="https://www.cronista.com/files/image/298/298782/5ffe0deb87a10.jpg" 
                        style="width:10000%; height:1000px  object-fit:cover; border-radius:20px; box-shadow: 0px 4px 15px rgba(0, 0, 0, 0.2);">
                    <p style="font-size: 18px; color: #666; margin-top: 8px;">Ciudad de las Artes y las Ciencias, Valencia </p>
                </div>
                """,
                unsafe_allow_html=True
            )

            # Métrica debajo de la imagen
            st.metric(label="Total de alojamientos analizados", value="9448", delta="Datos actualizados", help="Cantidad de registros en el dataset de Valencia Airbnb.")

    st.divider()

    with st.sidebar.markdown("**Sabías que...**"):
        st.markdown("""
        - Valencia es la **cuna de la paella**.  
        - Su **Lonja de la Seda** es Patrimonio de la Humanidad.  
        - Tiene uno de los **cascos históricos más grandes** de Europa.  
        """)

    # Widget 2: Checkbox
    check_box = st.sidebar.checkbox(label="Mostrar Dataset")

    if check_box:
        with st.container():
            st.markdown(
                """
                <div style="background-color: #ffe5b4; padding: 20px; border-radius: 15px; box-shadow: 0px 4px 12px rgba(0, 0, 0, 0.1);">
                    <h2 style="color: #d35400; text-align: center;">Dataset de Alojamiento en Valencia</h2>
                </div>
                """,
                unsafe_allow_html=True
            )

            st.markdown(
                """
                <style>
                div[data-testid="stTable"] table {
                    background-color: #ffe5b4;
                    color: #5c3d00;
                    border-radius: 10px;
                    overflow: hidden;
                    width: 100%;
                }
                thead tr {
                    background-color: #f7941d !important;
                    color: white !important;
                }
                tbody tr:hover {
                    background-color: #ffd580 !important;
                }
                </style>
                """,
                unsafe_allow_html=True
            )

            st.write(df)

            st.markdown("---")

#################################################################################################################
# CONTENIDO DE LA VISTA - Análisis Categórico y Numérico
if View == "Análisis Categórico y Numérico":
    st.title("Exploración de Categorías y Variables Numéricas")
    
    # Añadir pestañas para mejor organización
    tab1, tab2 = st.tabs(["Análisis Categórico", "Análisis Numérico"])
    
    with tab1:
        selected_cat_col = st.sidebar.selectbox("Selecciona una variable categórica", text_cols)
        
        # Crear tabla de frecuencias primero
        if selected_cat_col in df.columns:
            freq_table = df[selected_cat_col].value_counts().reset_index()
            freq_table.columns = [selected_cat_col, "Frecuencia"]
            
            # Calcular porcentajes
            total = freq_table["Frecuencia"].sum()
            freq_table["Porcentaje"] = (freq_table["Frecuencia"] / total * 100).round(2)
            
            # Límite para visualización
            n_top = st.sidebar.slider("Mostrar top categorías", 3, 15, 5)
            freq_table_top = freq_table.sort_values(by="Frecuencia", ascending=False).head(n_top)
        
            # Crear columnas para los gráficos
            col1, col2 = st.columns([2, 2])
        
            with col1:
                st.markdown("### Distribución en gráfico de pastel")
                fig_pie = px.pie(
                    freq_table_top, 
                    names=selected_cat_col, 
                    values="Frecuencia", 
                    title=f"Distribución de {selected_cat_col}",
                    color_discrete_sequence=px.colors.sequential.Oranges_r
                )
                fig_pie.update_traces(textposition='inside', textinfo='percent+label')
                fig_pie.update_layout(
                    legend=dict(orientation="h", yanchor="bottom", y=-0.3),
                    margin=dict(t=50, b=20, l=20, r=20)
                )
                st.plotly_chart(fig_pie, use_container_width=True)
        
            with col2:
                st.markdown("### Distribución en gráfico de barras")
                fig_bar = px.bar(
                    freq_table_top, 
                    x=selected_cat_col, 
                    y="Frecuencia",
                    color="Frecuencia",
                    color_continuous_scale="Oranges",
                    title=f"Frecuencia de {selected_cat_col}"
                )
                fig_bar.update_layout(
                    xaxis_title=selected_cat_col,
                    yaxis_title="Frecuencia",
                    coloraxis_showscale=False
                )
                st.plotly_chart(fig_bar, use_container_width=True)
        
            check_box = st.sidebar.checkbox(label="Mostrar Tabla de Frecuencias")
        
            if check_box:
                st.markdown("### Tabla de frecuencias de la columna seleccionada")
                st.dataframe(freq_table.style.background_gradient(cmap="Oranges", subset=["Frecuencia"]))
                
                # Añadir estadísticas adicionales para la variable categórica
                st.markdown("### Estadísticas adicionales")
                st.write(f"- Número de categorías únicas: {df[selected_cat_col].nunique()}")
                st.write(f"- Categoría más frecuente: {df[selected_cat_col].value_counts().index[0]}")
                st.write(f"- Porcentaje de valores nulos: {df[selected_cat_col].isna().mean() * 100:.2f}%")

    with tab2:
        # Generamos un cuadro de multi-selección (Y) para seleccionar variables a graficar
        numerics_vars_selected = st.sidebar.multiselect(label="Variables numéricas a graficar", options=numeric_cols)
        
        # Menu desplegable de opciones de la variable categórica seleccionada
        category_selected = st.sidebar.selectbox(label="Filtrar por tipo de habitación", options=unique_categories_room)
        
        # Despliegue de un line plot
        data = df[df['room_type'] == category_selected]
        
        if numerics_vars_selected:
            data_features = data[numerics_vars_selected]
            
            # Paleta de colores naranjas mejorada
            orange_palette = ['#FFA07A', '#FF6347', '#FFD700', '#FFA500']
            
            # Botones para diferentes tipos de gráficos
            plot_type = st.sidebar.radio(
                "Tipo de gráfico", 
                ["Line Plot", "Scatter Plot"]
            )
            
            if plot_type == "Line Plot":
                figure1 = px.line(
                    data_frame=data_features,
                    x=data_features.index,
                    y=numerics_vars_selected,
                    title=f'Variables numéricas para {category_selected}',
                    width=800,
                    height=500,
                    color_discrete_sequence=orange_palette,
                )
                figure1.update_layout(
                    xaxis_title="Índice",
                    legend_title="Variables",
                    hovermode="x unified"
                )
                st.plotly_chart(figure1, use_container_width=True)
                
            elif plot_type == "Scatter Plot" and len(numerics_vars_selected) >= 2:
                scatter_fig = px.scatter(
                    data_frame=data_features, 
                    x=numerics_vars_selected[0], 
                    y=numerics_vars_selected[1],
                    title=f"Relación entre {numerics_vars_selected[0]} y {numerics_vars_selected[1]}",
                    color_discrete_sequence=orange_palette,
                    opacity=0.7,
                    size_max=10,
                    trendline="ols" if st.checkbox("Mostrar línea de tendencia", value=True) else None
                )
                st.plotly_chart(scatter_fig, use_container_width=True) 
        else:
            st.info("Selecciona al menos una variable numérica para visualizar los datos.")
#################################################################################################################
# CONTENIDO DE LA VISTA - Análisis de correlaciones
if View == "Análisis de Correlaciones":
   
    
    st.title("Aplicación de  Regresión Lineal Simple y Múltiple")
    tab1, tab2, tab3 = st.tabs(["Heatmap","Regresión Lineal Simple", "Regresión Lineal Múltiple"])
    # ----------- TAB 1: heatmap -----------
    with tab1:
        # Asegúrate de que numeric_df tenga datos
        if 'numeric_df' in locals() and not numeric_df.empty:
            # Calcular la matriz de correlación
            corr = numeric_df.corr()
            Corr_Factors = abs(corr)
            orange_palette = px.colors.sequential.Oranges
            # Crear un heatmap interactivo con Plotly Express
            fig = px.imshow(
                Corr_Factors,
                color_continuous_scale=orange_palette,  # Escala de colores rojo-blanco-azul
                text_auto='.2f',                  # Mostrar valores con 2 decimales
                aspect='equal',                    # Celdas cuadradas
                labels=dict(color="Correlación")   # Etiqueta para la barra de color
            )
            
            # Mejorar el diseño y la visualización
            fig.update_layout(
                title='Matriz de Correlación',
                title_font=dict(size=22),
                width=800,                         # Ancho fijo para mejor visualización
                height=600,                        # Alto fijo para mejor visualización
                xaxis_title='',
                yaxis_title='',
                xaxis={'side': 'bottom'},
                margin=dict(l=50, r=50, t=80, b=50),
            )
            
            # Mejorar la visibilidad del texto
            fig.update_traces(
                textfont=dict(size=12, color='black', family='Arial Bold'),
                hoverinfo='all',
                hovertemplate='x: %{x}<br>y: %{y}<br>Correlación: %{z:.4f}<extra></extra>'
            )
            
            # Mostrar el gráfico interactivo en Streamlit
            st.plotly_chart(fig, use_container_width=True)
        
    # ----------- TAB 2: REGRESIÓN LINEAL SIMPLE -----------
    with tab2:
        st.subheader("Regresión Lineal Simple")
        col_x = st.sidebar.selectbox("Variable Independiente (X)", numeric_cols, key="rls_x")
        col_y = st.sidebar.selectbox("Variable Dependiente (Y)", numeric_cols, key="rls_y")

        X = df[[col_x]].values.reshape(-1, 1)
        y = df[col_y].values.reshape(-1, 1)

        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)

        st.write(f"**Coeficiente:** {model.coef_[0][0]:.4f}")
        st.write(f"**Intercepto:** {model.intercept_[0]:.4f}")

        scatter = px.scatter(df, x=col_x, y=col_y, color_discrete_sequence=['#FFA500'])
        scatter.add_scatter(x=df[col_x], y=y_pred.flatten(), mode='lines', name='Línea de Regresión',
                    line=dict(color='#FF7F0E', width=3))

        # Luego agregamos los puntos
        scatter.add_scatter(x=df[col_x], y=df[col_y], mode='markers', name='Datos reales',
                            marker=dict(color='#FFA500', size=6, opacity=0.7))

        st.plotly_chart(scatter)

    # ----------- TAB 3: REGRESIÓN LINEAL MÚLTIPLE -----------
    with tab3:
        st.subheader("Regresión Lineal Múltiple")
        
        selected_features = st.sidebar.multiselect("Variables Independientes", numeric_cols, key="rlm_x")
        target_var = st.sidebar.selectbox("Variable Dependiente", numeric_cols, key="rlm_y")

        if selected_features and target_var:
            X_mult = df[selected_features]
            y_mult = df[target_var]

            model_mult = LinearRegression()
            model_mult.fit(X_mult, y_mult)
            y_pred_mult = model_mult.predict(X_mult)

            st.write("**Coeficientes:**")
            coef_df = pd.DataFrame({
                'Variable': selected_features,
                'Coeficiente': model_mult.coef_
            })
            st.dataframe(coef_df)

            st.write(f"**Intercepto:** {model_mult.intercept_:.4f}")

            # Gráfico de valores reales vs predichos
            fig, ax = plt.subplots(figsize=(8, 6))
            ax.scatter(range(len(y_mult)), y_mult, color='gray', alpha=0.6, label='Reales')
            ax.scatter(range(len(y_pred_mult)), y_pred_mult, color='#FFA500', label='Predichos')

            # Líneas que conectan real con predicho
            np.random.seed(42)
            indices = np.random.choice(range(len(y_mult)), 10, replace=False)
            for i in indices:
                ax.plot([i, i], [y_mult.iloc[i], y_pred_mult[i]], color='#FFB347', alpha=0.5)

            ax.set_title('Regresión Lineal Múltiple: Reales vs Predichos')
            ax.set_xlabel('Índice de observación')
            ax.set_ylabel('Valor')
            ax.legend()
            st.pyplot(fig)


#################################################################################################################    
# CONTENIDO DE LA VISTA - Regresión Logística
if View == "Regresión Logística":
    
    st.sidebar.markdown("Regresión Logística")
    st.title("Aplicación de Regresión Logística")
    
    selected_features_log = st.sidebar.multiselect("Selecciona Variables Independientes", numeric_cols)

    # Paso 1: Variables object ya binarias (0 y 1 representables)
    binary_cols = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtype == 'object']

    # Paso 2: Detectar variables categóricas con entre 3 y 10 categorías (convertibles)
    candidates = [col for col in df.columns if df[col].dtype == 'object' and 2 < df[col].nunique() <= 60]

    # Mapeo: nombre visible en selector → variable binaria generada
    binarized_map = {}

    # Paso 3: Para cada variable candidata, crear UNA versión binaria
    for col in candidates:
        valores = df[col].dropna().unique()
        if len(valores) >= 2:
            valor_positivo = valores[0]
            valor_negativo = valores[1]
            nueva_columna = f"{col}_{valor_positivo}_vs_{valor_negativo}"
            
            # Solo considera 2 clases para crear binaria (descarta las demás si hay más de 2)
            df[nueva_columna] = df[col].apply(lambda x: 1 if x == valor_positivo else (0 if x == valor_negativo else np.nan))

            # Elimina filas con valores que no correspondan a las 2 categorías seleccionadas
            df = df[df[nueva_columna].isin([0, 1])]

            # Agrega el nombre original al selector y guarda su nueva columna
            binary_cols.append(col)
            binarized_map[col] = nueva_columna

    # Paso 4: Mostrar selector de variable dependiente SOLO UNA VEZ
    target_var_log = st.sidebar.selectbox("Selecciona Variable Dependiente", sorted(set(binary_cols)))

    # Paso 5: Obtener el nombre real de la columna binaria
    target_var_log_real = binarized_map.get(target_var_log, target_var_log)

    # Paso 6: Continuar si se seleccionaron variables independientes y dependiente
    df_log = df.dropna(subset=selected_features_log + [target_var_log_real])
    X = df_log[selected_features_log]
    y = df_log[target_var_log_real]

    if selected_features_log and target_var_log:
        df_log = df.dropna(subset=selected_features_log + [target_var_log])

        X = df_log[selected_features_log]
        y = df_log[target_var_log]

        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import StandardScaler
        from sklearn.linear_model import LogisticRegression
        from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=None)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model_log = LogisticRegression(max_iter=200)
        model_log.fit(X_train, y_train)

        y_pred_log = model_log.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred_log)
        matriz = confusion_matrix(y_test, y_pred_log)

        st.write(f"**Precisión del Modelo:** {accuracy * 100:.2f}%")

        st.write("**Coeficientes del Modelo:**")
        coef_log_df = pd.DataFrame({
            'Variable': selected_features_log,
            'Coeficiente': model_log.coef_[0]
        })
        st.dataframe(coef_log_df)

        # Matriz de confusión como mapa de calor
        # Obtener las etiquetas únicas
        labels = sorted(list(set(y_test.unique())))
        
        # Crear figura para el mapa de calor
        # Obtener etiquetas únicas
        labels = sorted(list(set(y_test.unique())))

        # Crear una matriz de confusión interactiva con Plotly
        fig = ff.create_annotated_heatmap(
            z=matriz,
            annotation_text=matriz,
            colorscale='Oranges',
            showscale=True
        )

        # Personalizar el diseño
        fig.update_layout(
            title='Matriz de Confusión',
            width=500,
            height=600,
            margin=dict(l=60, r=60, t=80, b=60)
        )

        # Asegurarse de que los valores sean claramente visibles
        fig.update_traces(
            texttemplate='%{z}',  # Mostrar valores tal cual
            #textfont={"size": 22, "color": "black"}  # Texto grande y negro
        )

        # Añadir bordes a las celdas para mejor definición
        for i in range(len(fig.data)):
            fig.data[i].update(showscale=True)

        # Mostrar la matriz de confusión interactiva
        st.plotly_chart(fig, use_container_width=True)

        
        # Mostrar el informe de clasificación
        st.write("**Informe de Clasificación:**")
        report = classification_report(y_test, y_pred_log, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        st.dataframe(report_df)

Overwriting app_Valencia.py
